# COMP5318 ASSIGNMENT 2 

**GROUP 144:**
- Aryan Bhatia : 490352056
- Mudit Malhotra : 490583269

**REFERENCES:**
- https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/
- https://towardsdatascience.com/cleaning-preprocessing-text-data-for-sentiment-analysis-382a41f150d6

## DATA ORGANIZATION AND CLEANING

### IMPORTING DATA

In [110]:
import pandas as pd 
import re

DATASET_COLUMNS=['target','ids','date','flag','user','text']
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('tweets.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)

dataset.keys()

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

### CLEANING DATA

**STEPS FOLLOWED FOR DATA CLEANING:**
- convert 4s to 1s (target) 
- drop duplicates
- lower case
- remove atsign
- remove urls 
- remove punctuation
- remove numbers 
- remove emojis
- remove stopwords
- lemmatization using spacy

In [111]:
# Converting 4s to 1s
data = dataset[['text','target']]
data['target'] = data['target'].replace(4,1)

In [112]:
# drop duplicates 
data = data.drop_duplicates(subset='text')

In [113]:
# lower case 
def lower_case(text):
    return text.lower()
data['text'] = data['text'].apply(lambda text: lower_case(text))

In [114]:
# remove urls
def remove_url(text):
    return re.sub('((www.[^s]+)|(https?://[^s]+))', ' ', text)
data['text'] = data['text'].apply(lambda text: remove_url(text))

In [115]:
# remove punctuations 
import string 

def remove_punctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

data['text'] = data['text'].apply(lambda text: remove_punctuations(text))

### FEATURE EXTRACTION
After data has been cleaned we perform feature extraction using the TF-IDF algorithm (term frequency-inverse document frequency)

### STORE DATASET

In [116]:
%store dataset

Stored 'dataset' (DataFrame)
